## Get data

In [2]:
%load_ext autoreload
%autoreload 2
import os, sys


import pandas as pd
import numpy as np
import plotly
from data_processing import process_data, clean_data, cluster_rental_location, generate_features

import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
pd.options.plotting.backend = "plotly"

DATA_DIR = './data'
train = os.path.join(DATA_DIR, 'train.csv')
test = os.path.join(DATA_DIR, 'test.csv')

train_final = os.path.join(DATA_DIR, 'train_with_mrt_mall_school.csv')
test_final = os.path.join(DATA_DIR, 'test_with_mrt_mall_school.csv')
train_final_df, test_final_df = pd.read_csv(train_final), pd.read_csv(test_final)
train_df, test_df = pd.read_csv(train), pd.read_csv(test)

train_df_cleaned, test_df_cleaned = clean_data(train_df), clean_data(test_df)
train_df_cleaned, test_df_cleaned = process_data(train_df_cleaned), process_data(test_df_cleaned, mode='test')
train_df_cleaned_final, test_df_cleaned_final = clean_data(train_final_df), clean_data(train_final_df)
train_df_cleaned_final, test_df_cleaned_final = process_data(train_df_cleaned_final), process_data(test_df_cleaned_final, mode='test')

train_df_cleaned, test_df_cleaned = generate_features(train_df_cleaned, test_df_cleaned)
train_df_cleaned_final, test_df_cleaned_final = generate_features(train_df_cleaned_final, test_df_cleaned_final)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Hypothesis

From the observation from the price trend, we formulate the hypothesis based on the correlation at "rent_approval_date" level, this is because with the increasing trend, pure scatter plot might not be easy to capture the relation ship between features and price. We might consider correlation with unit price (price per square meter)

All correlation should be analysed given a time period (group by rent_approval_date), to eliminate the trending effect. 

1. age is correlated with unit price
2. distance with mrt

In [3]:
train_df_cleaned_final.groupby(['rent_approval_date', 'region'])[['psqm', 'near_mrt_count']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='near_mrt_count', color='region')